In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
import cv2

In [ ]:
test_dir = "/kaggle/input/petfinder-pawpularity-score/train/"
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')

In [ ]:
os.chdir('/kaggle/working/yolov5') # install dependencies

In [ ]:
!python detect.py \
--weights /kaggle/input/ultralyticsyolov5aweights/yolov5x.pt \
--img 512\
--conf 0.1\
--iou 0.5\
--source $test_dir\
--name infer_fold \
--save-txt --save-conf --exist-ok >> /kaggle/working/log_yolo.txt
#--class 15 16 0 

In [ ]:
labelpaths = glob("runs/detect/infer_fold/labels/*")

predbox_ens = []
image_ids = []
score = []
label = []
x = []
y = []
w = []
h = []
index = []

for file_path in tqdm(labelpaths):
    image_id = file_path.split('/')[-1].split('.')[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]  
    for i,d in enumerate(data):
        index.append(len(data)-i)
        image_ids.append(image_id)
        score.append(d[1])
        label.append(d[0])
        x.append(d[2])
        y.append(d[3])
        w.append(d[4])
        h.append(d[5])

pred_df_yolo = pd.DataFrame({'Id':image_ids,'Index':index,'score':score,'label':label,
                             'x':x,'y':y,'w':w,'h':h
                            })
pred_df_yolo.to_csv("/kaggle/working/objectdetection.csv")

In [ ]:
pred_df_yolo

In [ ]:
"""
labelpaths = glob("runs/detect/infer_fold/labels/*")

predbox_ens = []
image_ids = []

score1 = []
label1 = []
w1 = []
h1 = []

score2 = []
label2 = []
w2 = []
h2 = []

score3 = []
label3 = []
w3 = []
h3 = []

for file_path in tqdm(labelpaths):
    image_id = file_path.split('/')[-1].split('.')[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]  
    image_ids.append(image_id)
    #get best 3 detection
    score1.append(data[0,1])
    label1.append(data[0,0])
    #x1.append(data[0,2])
    #y1.append(data[0,3])
    w1.append(data[0,4])
    h1.append(data[0,5])
    try:
        score2.append(data[1,1])
        label2.append(data[1,0])
        #x2.append(data[1,2])
        #y2.append(data[1,3])
        w2.append(data[1,4])
        h2.append(data[1,5])
    except:
        score2.append(0)
        label2.append(1)
        w2.append(0)
        h2.append(0)
        
    
    try:
        score3.append(data[2,1])
        label3.append(data[2,0])
        #x3.append(data[2,2])
        #y3.append(data[2,3])
        w3.append(data[2,4])
        h3.append(data[2,5])
    except:
        score3.append(0)
        label3.append(1)
        w3.append(0)
        h3.append(0)
        

pred_df_yolo = pd.DataFrame({'id':image_ids,
                             'score1':score1,
                             'label1':label1,
                             'w1':w1,
                             'h1':h1,
                             'score2':score2,
                             'label2':label2,
                             'w2':w2,
                             'h2':h2,
                             'score3':score3,
                             'label3':label3,
                             'w3':w3,
                             'h3':h3,
                            })
pred_df_yolo.to_csv("/kaggle/working/objectdetection.csv")
"""

In [ ]:
!zip -q /kaggle/working/yolopred.zip runs/detect/infer_fold/*
!rm -r /kaggle/working/yolov5